In [1]:
import cv2
import numpy as np

def find_edge(blurred_image, axis, reverse=False):
    """
    Find the edge of the Petri dish by scanning until a change in pixel intensity is detected.
    :param blurred_image: numpy array of the blurred image
    :param axis: 0 for horizontal, 1 for vertical
    :param reverse: False to scan from top/left, True to scan from bottom/right
    :return: index of the edge
    """
    sum_along_axis = np.sum(blurred_image, axis=axis)
    if reverse:
        sum_along_axis = sum_along_axis[::-1]
    
    # Threshold to detect edge, might need tuning based on actual image contrast and lighting
    threshold = sum_along_axis.max() * 0.05  # Adjusted lower due to blurring
    edge_idx = np.where(sum_along_axis < threshold)[0]
    if len(edge_idx) > 0:
        return edge_idx[0] if not reverse else blurred_image.shape[axis] - edge_idx[0]
    return 0

def crop_petri_dish(image_path):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian Blur
    blurred_image = cv2.GaussianBlur(image, (21, 21), 0)  # Kernel size and sigma can be adjusted

    # Find edges
    top_edge = find_edge(blurred_image, axis=0, reverse=False)
    bottom_edge = find_edge(blurred_image, axis=0, reverse=True)
    left_edge = find_edge(blurred_image, axis=1, reverse=False)
    right_edge = find_edge(blurred_image, axis=1, reverse=True)
    
    # Crop image
    cropped_image = image[top_edge:bottom_edge, left_edge:right_edge]
    
    # Save or display the cropped image
    cv2.imwrite('cropped_image.png', cropped_image)
    cv2.imshow('Cropped Image', cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Assuming a path to your image, you would call this function
# crop_petri_dish('path_to_your_image.png')
